Except as otherwise noted, the content of this page is licensed under the [Creative Commons Attribution 4.0 License](https://creativecommons.org/licenses/by/4.0/) (CC BY 4.0 Andrés Suárez), and code samples are licensed under the [MIT License](https://opensource.org/licenses/MIT) (Copyright 2020 Andrés Suárez). 

In [1]:
import numpy as np

In [2]:
def truss_element(E, A, ends):
    """" Generates the stiffness matrix of a bar
    
    Args:
        E (float): bar Young module [N/mm^2]
        A (float): bar transversal section [mm^2]
        ends (matrix): coordinates of the ends of the bar
        
    Returns:
        k (matrix): bar stiffness matrix [N/mm]
        
    """
    x1, y1 = ends[0,0], ends[0,1]
    x2, y2 = ends[1,0], ends[1,1]
    dx = x2-x1
    dy = y2-y1
    
    L = np.sqrt(dx**2 + dy**2)
    ls = dx/L
    ms = dy/L
    
    k = (E*A/L)*np.matrix([
        [ls**2, ls*ms, -ls**2, -ls*ms],
        [ls*ms, ms**2, -ls*ms, -ms**2],
        [-ls**2, -ls*ms, ls**2, ls*ms],
        [-ls*ms, -ms**2, ls*ms, ms**2]])
    
    return k

In [3]:
def global_stiffness_matrix(Ee, Ae, coord, nd, lm):
    """" Generates the global stiffness matrix of the structure
    
    Args:
        Ee (array): bars Young modulus [N/mm^2]
        Ae (array): bars cross sections [mm^2]
        coord (matrix): coordinates of the bars ends [mm]
        nd (int): degrees of freedom
        lm (array): nodal axis
        
    Returns:
        kg (matrix): global stiffness matrix [N/mm]
        
    """
    K = np.zeros((nd,nd))
    for i in range(ne):
        ke = truss_element(Ee[i], Ae[i], coord[np.ravel(en[i,:]),:])
        ix = np.ravel(lm[i,:])
        K[np.ix_(ix,ix)] = K[np.ix_(ix,ix)] + ke
    return K

In [4]:
def nodal_displacement(K, F, B, R):
    """" Estimates the nodal displacements
    
    Args:
        K (matrix): global stiffness matrix [N/mm^2]
        F (array): nodal loads [N]
        B (array): nodal axis with restrictions
        R (array): fixed displacements of the restricted nodal axis [mm]
        
    Returns:
        d (array): nodal displacements [mm]
        
    """
    dof = len(F)
    d = np.zeros([dof,1])
    free = np.setxor1d(np.arange(dof), B)
    Kf = K[np.ix_(free, free)]
    Rf = F[free] - np.dot(K[np.ix_(free,B.ravel())], R)
    sol = np.dot(np.linalg.inv(Kf), Rf)
    d[B.T] = R
    d[free] = sol
    return d

In [5]:
# geometry
ne = 3                     # number of elements
nn = 3                     # number of nodes
nd = 6                     # number of degrees of freedom
coord = 1000*np.matrix('0 0; 4 0; 0 3')        # nodal coordinates
en = np.matrix('0 1; 1 2; 0 2')                # bar connectivity
lm = np.matrix('[0 1 2 3; 2 3 4 5; 0 1 4 5]')  # nodal axis

# materials and sections
E1 = 70e3                     # N/mm^2
E2 = 210e3                    # N/mm^2
A1 = 3e3                      # mm^2
A2 = 2e3                      # mm^2
Ee = np.array([E1, E2, E1])   # size = number of bars
Ae = np.array([A1, A2, A1])   # size = number of bars

# loads
P = 150000;                    # N
alpha = 30*np.pi/180           # radians
F = np.zeros([nd, 1])
F[2] = -P*np.sin(alpha) 
F[3] = -P*np.cos(alpha)

# restrictions
B = np.array([[0, 1, 4]]).T    # Nodal axis...
R = np.array([[0, 0, 0]]).T    # ...with restrictions

In [6]:
# nodal displacement
K = global_stiffness_matrix(Ee, Ae, coord, nd, lm)
d = nodal_displacement(K, F, B, R)
d.reshape(-1,2)

array([[  0.        ,   0.        ],
       [ -4.72771582, -12.45515075],
       [  0.        ,  -1.85576872]])